In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
from tensorflow.keras.layers import Conv2D

from preprocessing.data.imagenet_labels import imagenet_labels

In [3]:
from models.ResBlock import ResBlock

In [4]:
from modules.TFR_load import TFR_load

In [5]:
# Hyper params

BATCH_SIZE     = 32
NUM_OF_CLASS   = 10
LEARNING_RATE  = 1e-5
COSINE_DECAY   = True
NUM_TRAIN_DATA = 60000
NUM_TEST_DATA  = 10000
EPOCH = 100

TR_STEPS_PER_EPOCH = NUM_TRAIN_DATA//BATCH_SIZE
TE_STEPS_PER_EPOCH = NUM_TEST_DATA//BATCH_SIZE

In [6]:
import tensorflow_datasets as tfds

ds_name = 'mnist'
builder = tfds.builder(ds_name)

tr_ds, te_ds = builder.as_dataset(split = ['train', 'test'], shuffle_files = True)

tr_ds = tr_ds.batch(BATCH_SIZE)
print(len(tr_ds))
te_ds = te_ds.batch(BATCH_SIZE)
print(len(te_ds))

1875
313


In [7]:
out_ch = [16, 32, 64, 128]
filter_size = [(5,5),(5,5),(3,3),(3,3),(3,3)]

In [8]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.conv_list = []
        for i in range(len(out_ch)):
            conv = Conv2D(
                out_ch[i],
                filter_size[i],
                strides = (1, 1),
                padding='same',
                activation = "relu"
            )
            self.conv_list.append(conv)
        
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(NUM_OF_CLASS, activation = 'softmax')
        
    def call(self, x):
        for conv in self.conv_list:
            x = conv(x)
        
        x = self.flatten(x)
        x = self.dense(x)
        
        return x

model = MyModel()

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=1e-08)
loss = tf.keras.losses.CategoricalCrossentropy()

for _ in range(EPOCH):
    for example in tr_ds:
        images, labels = example["image"], example["label"]
        with tf.GradientTape() as tape:
            logits = model(images / 255)
            loss_value = loss(logits, tf.one_hot(labels, 10))
        print(loss_value.numpy())
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

14.498497
14.499971
14.507822
14.498472
14.498627
14.498554
14.5043545
14.496616
14.495899
14.49454
14.49279
14.498326
14.493813
14.49069
14.482264
14.481279
14.485979
14.480051
14.485617
14.4810295
14.469057
14.467419
14.473341
14.478085
14.471717
14.46497
14.47719
14.464857
14.459857
14.449945
14.46701
14.468925
14.458124
14.450378
14.457155
14.439894
14.453739
14.445104
14.435974
14.460199
14.441343
14.424635
14.451548
14.420361
14.435019
14.423004
14.426293
14.411774
14.428474
14.442386
14.406046
14.400938
14.406464
14.40624
14.4419985
14.418413
14.411089
14.416447
14.403512
14.424206
14.381715
14.383504
14.396574
14.375876
14.362619
14.364236
14.38533
14.372295
14.326506
14.402298
14.365548
14.323158
14.356208
14.364284
14.310101
14.37488
14.364662
14.312501
14.395132
14.382542
14.408834
14.23085
14.364309
14.340102
14.282401
14.346558
14.322604
14.265432
14.323924
14.212643
14.261453
14.274958
14.321079
14.22052
14.130499
14.251791
14.252392
14.233708
14.309931
14.256477
14.21366

4.0348873
3.9656518
1.9286239
1.780919
1.9593494
3.03852
2.876813
2.3182845
1.6727172
2.2113273
1.507406
2.2400224
4.4427176
5.0266967
2.8371744
5.411824
1.235043
2.6701546
5.1615753
2.7629135
4.0734425
3.1402872
4.7018538
6.0894704
2.9050303
4.8678036
1.8646302
3.3531914
2.2255003
4.3695955
4.2940154
2.2782376
2.7455337
4.200536
1.8313267
4.8936205
5.4139376
4.2813644
4.0525208
3.371759
1.5257938
2.6873326
4.716563
5.8216767
3.2168107
2.7362576
3.617437
3.0289617
3.6576314
4.239883
2.6080875
4.620728
3.81236
1.4936001
4.0308213
2.8369577
3.225259
4.9401846
1.2097986
5.3589826
1.2849075
4.391848
3.7049766
5.0326605
3.0528572
2.2098796
2.4209166
2.8881478
5.5410953
3.2814429
3.915627
4.212105
5.010782
2.8475966
6.2194347
3.749474
4.4706926
6.501046
2.8502004
3.2589636
3.6068265
4.7773666
1.2795787
3.6352472
3.0285544
4.3636475
5.016015
3.1496823
3.8205657
3.4284816
5.116411
4.238994
1.825917
3.1377573
4.3119087
4.4765735
3.1004772
1.1212068
4.7447586
1.5711107
3.199688
2.1891594
2.96333

1.1824789
3.7889395
1.2730988
4.8209715
3.2488775
3.6626148
2.7418213
0.9421896
3.7734551
3.011835
3.5958338
3.7459486
2.3450716
3.3645077
3.8155842
2.6145902
2.5583725
4.4543543
2.6799512
3.6041875
2.3772788
4.184779
2.3438106
2.0401678
4.6366267
5.1381483
2.0223265
1.0604658
2.608902
1.2704542
2.6919475
1.5701452
2.1075802
4.5640063
1.5669847
3.3953712
3.355044
2.1143126
2.6888433
3.9080906
2.940915
2.980166
4.141647
2.5241618
1.7232332
3.1997552
2.1334138
2.4981108
1.665257
3.056054
2.150285
3.5386403
3.1265488
3.893234
2.022336
3.6585302
4.515179
3.5482013
2.009461
2.2312326
1.732385
1.9161568
2.1123872
2.8252268
4.2085156
4.1767464
2.2059424
3.7568388
3.4712152
1.9585214
2.5121603
4.0581474
2.1757247
2.0313416
1.624394
3.323651
1.4627005
3.3857465
4.476855
2.3144
2.758696
3.0077546
2.667171
1.9092648
4.640619
3.2605944
3.0813742
3.4996824
4.2913184
2.2801833
2.6664822
2.8347626
3.1067793
3.5536265
3.942344
2.2724605
2.5295634
4.6104593
3.0324323
2.3652172
3.767864
2.6781645
3.4291

2.1223671
2.0930476
1.4459499
1.6428409
0.9039031
1.5451417
3.0321999
1.9393969
2.4022832
1.8696265
0.62616587
3.9002123
2.3800364
1.4413749
2.697472
1.2120161
2.3699918
1.8569024
2.2933648
0.82176846
1.1075459
1.4562304
2.9163651
1.1583307
2.4246178
2.5320692
2.3324575
1.6380873
1.3772545
0.54918987
2.0712333
1.3089168
2.164497
1.6068563
1.2687786
1.3063874
1.0361737
2.2992265
1.4289477
1.7393913
1.8742034
3.4175682
2.4482827
1.3162811
2.020192
2.3533986
2.0390856
1.2317739
3.1168735
1.6314198
0.8768156
1.3100488
2.0170393
0.80494636
0.26937136
0.4618559
1.6687813
2.0740085
0.82266724
1.9746852
1.0237328
2.2455316
2.3330705
1.5561135
1.250494
2.1208136
2.4091177
1.8505802
2.1084714
1.3425854
2.4660811
0.8683433
1.803968
1.0609934
1.6611903
2.3154805
3.4677656
2.0856807
1.0503962
0.8748703
4.0234118
1.9927626
1.5073208
0.5557306
3.6305497
1.8692155
2.3536196
2.1544147
2.34018
1.1416682
1.8773928
1.6221168
2.9976735
0.9847815
1.6412634
3.6412745
2.0262089
0.70993614
1.8701341
1.3386927


0.22271666
2.6283133
1.6590973
2.1423774
1.3857517
3.3828592
2.373914
1.0933919
3.1411452
1.8580241
2.747277
2.814612
1.6117935
3.3456926
0.56322366
0.60190386
1.1261834
1.416109
1.0637344
0.53989637
1.7386734
0.4042135
1.0278599
1.3804762
1.3437773
2.1781728
1.4495875
2.713494
1.0403558
1.2015254
2.9937935
1.4500318
2.2455878
0.8122206
2.4270473
0.68161017
2.8837128
0.53999174
0.58949447
1.4900838
1.1128547
0.57870555
2.544304
1.0871546
1.4244256
2.3228602
1.9560628
2.3022842
1.2506777
1.4542055
2.0856605
2.4886708
2.2463546
1.0887371
2.44409
1.3846428
2.0251281
1.064753
0.72962594
1.2465746
1.8742241
1.2933931
2.0151215
2.2409372
3.3293846
1.7494085
1.6407067
1.3042393
1.8734657
2.227164
0.10233079
0.8616668
1.0842351
1.5818944
2.950757
1.5022588
1.9417193
3.3147902
2.4363484
1.4775074
0.86313516
1.9657322
1.5824829
0.6041663
1.9095417
1.8451831
0.25783634
1.5142821
2.0244198
0.28005087
1.6473806
1.9984179
2.5039659
2.038778
0.78838634
1.5495465
1.8931524
1.6179097
2.1132867
1.603427

...

1.3019454
0.009674859
2.6936169
0.100412734
0.48660025
0.07229217
0.5338839
0.5081165
0.32501596
0.05199084
0.0043289047
1.3314328
0.5050326
0.16244714
0.73345494
1.0140661
0.6568153
1.1543937
0.9711764
0.0042731636
0.7968434
0.15102014
1.4681956
1.7668984
0.04363176
0.562483
1.7033556
2.1196613
0.60106915
0.5617802
0.5108765
0.5418792
0.35383093
1.1883119
0.79396653
0.39814058
0.022672702
0.042951405
0.16015361
0.81638217
0.9364578
0.6433667
0.030037772
0.12495507
0.5855175
0.89078355
0.056685157
0.6156413
0.29226276
0.061962742
0.25325182
0.66612583
0.009407984
0.120697
0.068341866
0.620557
0.34425244
1.745621
0.6140774
1.0728045
1.9515388
0.6811408
0.8205841
0.32934776
0.6027653
1.0454905
1.3175497
0.99487615
0.5617833
1.6526468
0.012961542
0.940716
0.043878257
1.0988846
0.53995794
1.2661536
1.0433121
0.018443935
0.49369264
0.55342275
0.048702676
0.041673783
0.45743424
0.006734046
0.30600595
1.312321
0.8272853
0.6400555
0.81898564
1.6966307
0.5315681
0.10001913
0.40777454
0.30371386

0.036723938
0.5824878
0.31323978
0.528879
0.53879595
0.41699702
0.6676231
0.080031954
0.98597777
1.1816936
0.268666
1.8906752
0.16959122
0.5492979
0.90623456
0.102135934
0.9479996
0.5122702
0.8816246
0.5154774
0.9792999
0.004945089
0.62697345
0.92058897
0.835005
0.37034237
0.5255475
0.4328298
0.9692219
0.32329163
0.19089548
0.04880813
0.9188704
1.9057993
0.0016483865
0.032686614
0.07534279
0.95199645
0.040731613
0.5324153
0.16021231
1.0309999
0.5100264
0.119452834
0.032145
0.55452526
0.08053767
0.82226604
0.14152703
0.5494727
0.54297763
1.2328796
1.2506909
0.51091635
0.32431078
0.05716312
0.9958147
0.9879911
0.16836122
0.03738452
0.34460798
2.5553937
0.9332807
1.1048923
0.03690886
0.99293244
0.03255039
0.8187314
0.38348505
0.46864614
0.6017868
0.38645384
0.018752579
1.4616861
1.1155939
0.0011556661
0.5125367
1.5133219
0.12318809
0.08588618
0.50186944
0.5015758
0.0042684292
1.2959368
0.51416993
0.52747273
0.091371104
1.3803174
0.50551695
0.29152504
0.5048273
0.60106707
0.48528373
0.1151

0.0071333367
0.13705005
0.0960012
0.6643948
1.239414
0.037411
1.021123
0.5357671
0.56166655
1.5589697
2.2539628
1.0389944
0.35999578
0.5234746
1.1705029
1.16527
2.8670003
0.5450299
0.065017834
0.3871468
0.18699308
0.0013323227
1.5637221
0.7802557
1.3768055
0.52983505
0.65479636
0.9372901
0.023081709
0.99462306
0.0013833395
0.5293685
0.4318709
0.5291902
1.5538781
0.49305418
0.004572802
0.7087467
0.039140422
0.76935077
0.5051483
0.6884157
0.056007124
1.5364454
0.5246704
0.83385056
1.6814792
0.2717398
1.2537006
1.1782401
0.19737953
1.0433986
0.45749623
1.3419896
0.026742358
0.6145723
0.056999683
0.5431224
0.90165716
0.55857337
0.10649759
0.018258607
0.6596985
0.16979297
1.4979196
0.5239382
0.4936249
0.004490114
0.07944879
0.39430398
0.051110774
0.63313836
0.93660957
0.12769818
0.4851043
0.09744145
0.18001549
1.3729498
0.5988626
0.08076504
1.0379329
0.038835682
0.36777693
0.9327197
0.0024235572
0.24692944
0.5676898
1.0816281
1.8961835
0.01138426
0.0011058801
1.0125964
0.034325805
0.0250996

1.5375031
1.003854
0.97004855
1.0190151
0.7947965
0.008946452
0.52980757
0.015659744
2.0510168
0.023551114
0.05976393
1.3599796
0.6415423
0.50907487
0.52101696
0.017423268
0.13442741
0.08091183
0.81944495
0.3285659
0.012221637
0.0049637407
0.4378397
1.4342184
0.10237642
1.0347815
0.001496572
0.052991237
0.5393642
0.029280916
0.98247635
0.048452437
0.69201136
0.10355598
0.13142784
1.0301653
0.9824272
1.0666502
0.011108279
1.1005859
0.72039074
0.25704286
0.07320022
0.09652277
0.014764068
0.7406215
0.57123715
1.4456388
0.0035584297
0.48253968
0.00080599356
0.090965815
0.030317534
0.16969894
0.7109039
1.5385191
0.013795291
0.021765951
0.66946757
0.38720283
0.5093245
0.6966174
0.90683186
0.30495837
0.95999277
0.13270369
0.10379227
0.037758823
0.12732014
0.037878774
0.13526812
0.0075723026
0.117226616
1.2219799
0.42166352
0.5473931
0.17055178
0.11881958
0.23215815
1.1514323
0.30056217
0.79499686
0.7120418
0.038174916
0.5095506
0.01172691
1.5911068
1.2457082
0.03610848
0.97042584
0.49853286
0

KeyboardInterrupt: 